In [114]:
import sys
import os

import warnings

import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.data import Dataset
from PIL import Image
from image import *
import tensorflow.image
from keras.optimizers import SGD, Adam
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Conv2D 

import numpy as np
import json
import cv2
import dataset
import time


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

tf.enable_eager_execution()

In [63]:
train_list = []
with open('train_data_list.txt', 'r') as f:
    for line in f:
        inner_list = [elt.strip() for elt in line.split(',')]
        train_list.append(inner_list)
train_list = [val.replace('\'','') for val in train_list[0]]

In [64]:
test_list = []
with open('test_data_list.txt', 'r') as f:
    for line in f:
        inner_list = [elt.strip() for elt in line.split(',')]
        test_list.append(inner_list)
test_list = [val.replace('\'','') for val in test_list[0]]

In [80]:
# import image
def load_data(paths, train = True):
    for img_path in paths:
        gt_path = img_path.decode("utf-8").replace('.jpg','.h5').replace('images','ground-truth')
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels = 3)
        img = tf.image.convert_image_dtype(img, tf.float32)

        gt_file = h5py.File(gt_path, 'r')
        target = np.asarray(gt_file['density'])

        target = cv2.resize(target,(int(target.shape[1]/8),int(target.shape[0]/8)),interpolation = cv2.INTER_CUBIC)*64

    
        yield (img, target)

In [87]:
train_dataset = tf.data.Dataset.from_generator(
                        load_data, args = [train_list] , output_types = (tf.float32, tf.float32), output_shapes = ((None,None,3), (None,None)))
train_dataset = train_dataset.shuffle(100000)


In [88]:
test_dataset = tf.data.Dataset.from_generator(
                   load_data, args = [test_list] , output_types = (tf.float32, tf.float32), output_shapes = ((None,None,3), (None,None)))

# Define function

In [39]:
batch_size = 100
epochs = 15
IMG_HEIGHT = 255
IMG_WIDTH = 255

In [40]:
loss_object = tf.keras.losses.MeanSquaredError()

In [41]:
optimizer = Adam(2e-4)

In [42]:
def loss_fn(model, input_image, gt_image):

    output = model(input_image)
    pred = np.sum(output) 
    
    ground_truth = tf.cast(ground_truth, tf.float32)
    ground_truth = np.sum(ground_truth)
    
    # mean squared error
    loss = loss_object(ground_truth, pred)
    return loss.numpy()

In [103]:
# check how it shows
for img, tar in train_dataset.take(1):
    print(img)

/Users/seungmi/anaconda3/envs/pyDL/lib/python3.6/site-packages/ipykernel_launcher.py:9: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  if __name__ == '__main__':


tf.Tensor(
[[[0.28235295 0.32941177 0.28235295]
  [0.2392157  0.28627452 0.2392157 ]
  [0.20784315 0.25490198 0.21568629]
  ...
  [0.69803923 0.81568635 0.8470589 ]
  [0.69803923 0.81568635 0.8470589 ]
  [0.69411767 0.8117648  0.8431373 ]]

 [[0.23529413 0.28235295 0.23529413]
  [0.20392159 0.2509804  0.20392159]
  [0.1764706  0.22352943 0.18431373]
  ...
  [0.69803923 0.81568635 0.8470589 ]
  [0.69803923 0.81568635 0.8470589 ]
  [0.69411767 0.8117648  0.8431373 ]]

 [[0.20000002 0.24705884 0.20784315]
  [0.17254902 0.21960786 0.18039216]
  [0.14509805 0.19215688 0.15294118]
  ...
  [0.69803923 0.81568635 0.8470589 ]
  [0.69803923 0.81568635 0.8470589 ]
  [0.69411767 0.8117648  0.8431373 ]]

 ...

 [[0.49411768 0.5764706  0.5803922 ]
  [0.50980395 0.5921569  0.59607846]
  [0.52156866 0.6039216  0.60784316]
  ...
  [0.3803922  0.4431373  0.4431373 ]
  [0.38431376 0.44705886 0.44705886]
  [0.38431376 0.44705886 0.44705886]]

 [[0.50980395 0.5921569  0.59607846]
  [0.5137255  0.59607846 0

In [107]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(None, None, 3))

In [108]:
#model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)


In [123]:
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME', 
                                  input_shape=(None, None,3)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(1))
    return model

In [124]:
model = create_model()

In [127]:
for img, target in train_dataset.take(1):
        output = model(np.expand_dims(img,0))
        

/Users/seungmi/anaconda3/envs/pyDL/lib/python3.6/site-packages/ipykernel_launcher.py:9: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  if __name__ == '__main__':


-823.663
1018630.0
576.0004


In [ ]:
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in train_dataset:
        grads = grad(model, images, labels)                
        optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss + loss
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:        
        acc = evaluate(model, images, labels)        
        avg_test_acc = avg_test_acc + acc
        test_step += 1    
    avg_test_acc = avg_test_acc / test_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))
    
print('Learning Finished!')